In [1]:
import re
import csv
import time
from playwright.async_api import async_playwright
from parsel import Selector

In [2]:
def extract_lat_long(url):
    patterns = [
        r'@(-?\d+\.\d+),(-?\d+\.\d+)',
        r'destination=(-?\d+\.\d+)/(-?\d+\.\d+)',
        r'query=(-?\d+\.\d+),(-?\d+\.\d+)'
    ]
    lat, long = None, None
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            lat, long = match.groups()
            lat, long = float(lat), float(long)
            break
    return lat, long



In [3]:
urls_data = []

with open('no-google-map-url.csv', 'r') as f: 
    reader = csv.DictReader(f)
    for row in reader:
        _01_category = row['_01_category']
        _02_state = row['_02_state']
        _03_branch_name = row['_03_branch_name']
        _04_address = row['_04_address']
        # _05_latitude
        # _06_longitude
        _07_bank_type = row['_09_bank_type']
        _08_mother_bank_name = row['_11_mother_bank_name']

        _07_google_map_url = row['_07_google_map_url']

        urls_data.append({
            "_01_category": _01_category,
            "_02_state": _02_state,
            "_03_branch_name": _03_branch_name,
            "_04_address": _04_address,
            "_07_bank_type": _07_bank_type,
            "_08_mother_bank_name": _08_mother_bank_name,
            "_07_google_map_url": _07_google_map_url
        })


In [4]:
data = []
count = 1

for row in urls_data:
    _01_category = row['_01_category']
    _02_state = row['_02_state']
    _03_branch_name = row['_03_branch_name']
    _04_address = row['_04_address']
    _07_bank_type = row['_07_bank_type']
    _08_mother_bank_name = row['_08_mother_bank_name']
    _07_google_map_url = row['_07_google_map_url']


    _05_latitude, _06_longitude = extract_lat_long(_07_google_map_url)


    meta_dic = {
        "_01_category": _01_category,
        "_02_state": _02_state,
        "_03_branch_name": _03_branch_name,
        "_04_address": _04_address,
        "_05_latitude": _05_latitude,
        "_06_longitude": _06_longitude,
        "_07_bank_type": _07_bank_type,
        "_08_mother_bank_name": _08_mother_bank_name,
    }

    data.append(meta_dic)

    print(f"\rProcessing profile {count}...", end="")
    count += 1
    

Processing profile 268...

In [5]:
csv_filename = 'lat-long-3.csv'

field_names = data[0].keys() if data else []
with open(csv_filename, 'a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(data)